## Jester Jokes Content based Rec. Sys
## Import Libraries

In [1]:
import os
import re
import nltk
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#nltk.download()

## Upload the files

In [2]:
ratings_df1 = pd.read_excel('jester-data-1.xls', header=None)
ratings_df2 = pd.read_excel('jester-data-2.xls', header=None)
ratings_df3 = pd.read_excel('jester-data-3.xls', header=None)

list_of_jokes = []
html_files = []
rootdir = os.getcwd()

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #print(os.path.join(subdir, file))
        filepath = subdir + os.sep + file
        if filepath.endswith(".html"):
            html_files.append(filepath)
html_files.sort(key=lambda w: int(w[47:].split('.')[0]))
for files in html_files:
    soup = BeautifulSoup(open(files), 'html')
    for i in soup.find_all('td', class_=False):
        if len(i)>1:
            i = i.text.split('\n')
            jokes = [joke for joke in i if len(joke)>1]
            jokes = ' '.join(jokes)
            list_of_jokes.append(jokes)
        else:pass

df_jokes = pd.DataFrame(list_of_jokes, columns = ['Jokes'])
df_jokes.index = np.arange(1, len(df_jokes) + 1)
df_jokes['joke_id'] = df_jokes.index
df_jokes

,Jokes,joke_id
1,"A man visits the doctor. The doctor says ""I ha...",1
2,This couple had an excellent relationship g...,2
3,Q. What's 200 feet long and has 4 teeth? A...,3
4,Q. What's the difference between a man and ...,4
5,Q.\tWhat's O. J. Simpson's Internet address...,5
...,...,...
96,Two attorneys went into a diner and ordered tw...,96
97,A teacher is explaining to her class how diffe...,97
98,Age and Womanhood 1. Between the ages of 13 an...,98
99,A bus station is where a bus stops. A train st...,99


## MERGING RATES AS ONE SINGLE TABLE

In [3]:
ratings_df1 = ratings_df1.drop([0], 1)
ratings_df1 = ratings_df1.T
ratings_df2 = ratings_df2.drop([0], 1)
ratings_df2 = ratings_df2.T
ratings_df3 = ratings_df3.drop([0], 1)
ratings_df3 = ratings_df3.T
ratings_df1

,0,1,2,3,4,5,6,7,8,9,...,24973,24974,24975,24976,24977,24978,24979,24980,24981,24982
1,-7.82,4.08,99.00,99.00,8.50,-6.17,99.00,6.84,-3.79,3.01,...,1.80,99.00,99.00,1.36,99.00,0.44,9.13,99.00,99.00,2.43
2,8.79,-0.29,99.00,8.35,4.61,-3.54,99.00,3.16,-3.54,5.15,...,-9.42,-9.22,8.20,1.75,99.00,7.43,-8.16,99.00,99.00,2.67
3,-9.66,6.36,99.00,99.00,-4.17,0.44,99.00,9.17,-9.42,5.15,...,-8.20,99.00,99.00,2.38,99.00,9.08,8.59,99.00,99.00,-3.98
4,-8.16,4.37,99.00,99.00,-5.39,-8.50,99.00,-6.21,-6.89,3.01,...,99.00,99.00,99.00,1.84,99.00,2.33,9.08,99.00,99.00,4.27
5,-7.52,-2.38,9.03,1.80,1.36,-7.09,8.59,-8.16,-8.74,6.41,...,-8.54,5.97,8.79,4.66,7.67,3.20,0.87,-7.77,-9.71,-2.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,99.00,-2.14,99.00,99.00,1.55,-3.35,2.33,-3.35,-0.29,99.00,...,99.00,99.00,8.93,-0.34,99.00,9.03,-8.20,99.00,99.00,99.00
97,-5.63,3.06,99.00,99.00,3.11,0.05,99.00,2.14,-0.29,99.00,...,99.00,99.00,99.00,9.32,99.00,6.55,-7.23,99.00,99.00,99.00
98,99.00,0.34,99.00,99.00,6.55,-9.08,99.00,-0.05,-0.29,99.00,...,99.00,99.00,9.13,9.17,99.00,8.69,-8.59,99.00,99.00,99.00
99,99.00,-4.32,99.00,99.00,1.80,-5.05,99.00,1.31,-3.40,99.00,...,-3.74,99.00,99.00,8.88,99.00,8.79,9.13,99.00,99.00,99.00


## JOIN RATINGS TO JOKES

In [4]:
ratings_df = pd.merge(ratings_df1, ratings_df2, right_index=True, left_index=True)
ratings_df = pd.merge(ratings_df, ratings_df3, right_index=True, left_index=True)
ratings_df.columns = range(1, 1 + len(ratings_df.columns))
ratings_df = ratings_df.add_prefix('User')
jokes_rate_df = pd.merge(df_jokes, ratings_df, right_index=True, left_index=True)
jokes_rate_df

,Jokes,joke_id,User1,User2,User3,User4,User5,User6,User7,User8,...,User73412,User73413,User73414,User73415,User73416,User73417,User73418,User73419,User73420,User73421
1,"A man visits the doctor. The doctor says ""I ha...",1,-7.82,4.08,99.00,99.00,8.50,-6.17,99.00,6.84,...,99.0,99.0,99.00,7.14,6.46,99.00,99.00,99.00,99.00,99.00
2,This couple had an excellent relationship g...,2,8.79,-0.29,99.00,8.35,4.61,-3.54,99.00,3.16,...,99.0,99.0,99.00,99.00,8.69,99.00,99.00,99.00,99.00,99.00
3,Q. What's 200 feet long and has 4 teeth? A...,3,-9.66,6.36,99.00,99.00,-4.17,0.44,99.00,9.17,...,99.0,99.0,99.00,3.88,9.08,99.00,99.00,99.00,99.00,99.00
4,Q. What's the difference between a man and ...,4,-8.16,4.37,99.00,99.00,-5.39,-8.50,99.00,-6.21,...,99.0,99.0,99.00,1.12,3.79,99.00,99.00,99.00,99.00,99.00
5,Q.\tWhat's O. J. Simpson's Internet address...,5,-7.52,-2.38,9.03,1.80,1.36,-7.09,8.59,-8.16,...,5.1,99.0,8.88,99.00,6.46,5.78,5.24,-3.93,9.42,0.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Two attorneys went into a diner and ordered tw...,96,99.00,-2.14,99.00,99.00,1.55,-3.35,2.33,-3.35,...,99.0,99.0,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00
97,A teacher is explaining to her class how diffe...,97,-5.63,3.06,99.00,99.00,3.11,0.05,99.00,2.14,...,99.0,99.0,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00
98,Age and Womanhood 1. Between the ages of 13 an...,98,99.00,0.34,99.00,99.00,6.55,-9.08,99.00,-0.05,...,99.0,99.0,99.00,99.00,-0.24,99.00,99.00,99.00,99.00,99.00
99,A bus station is where a bus stops. A train st...,99,99.00,-4.32,99.00,99.00,1.80,-5.05,99.00,1.31,...,99.0,99.0,99.00,9.81,99.00,99.00,99.00,99.00,99.00,99.00


## CHANGE 99 RATINGS TO NaN VALUES

In [5]:
jokes_rate_df = jokes_rate_df.replace(99, np.nan)
jokes_rate_df.iloc[98,1] = 99
jokes_rate_df

,Jokes,joke_id,User1,User2,User3,User4,User5,User6,User7,User8,...,User73412,User73413,User73414,User73415,User73416,User73417,User73418,User73419,User73420,User73421
1,"A man visits the doctor. The doctor says ""I ha...",1.0,-7.82,4.08,NaN,NaN,8.50,-6.17,NaN,6.84,...,NaN,NaN,NaN,7.14,6.46,NaN,NaN,NaN,NaN,NaN
2,This couple had an excellent relationship g...,2.0,8.79,-0.29,NaN,8.35,4.61,-3.54,NaN,3.16,...,NaN,NaN,NaN,NaN,8.69,NaN,NaN,NaN,NaN,NaN
3,Q. What's 200 feet long and has 4 teeth? A...,3.0,-9.66,6.36,NaN,NaN,-4.17,0.44,NaN,9.17,...,NaN,NaN,NaN,3.88,9.08,NaN,NaN,NaN,NaN,NaN
4,Q. What's the difference between a man and ...,4.0,-8.16,4.37,NaN,NaN,-5.39,-8.50,NaN,-6.21,...,NaN,NaN,NaN,1.12,3.79,NaN,NaN,NaN,NaN,NaN
5,Q.\tWhat's O. J. Simpson's Internet address...,5.0,-7.52,-2.38,9.03,1.80,1.36,-7.09,8.59,-8.16,...,5.1,NaN,8.88,NaN,6.46,5.78,5.24,-3.93,9.42,0.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Two attorneys went into a diner and ordered tw...,96.0,NaN,-2.14,NaN,NaN,1.55,-3.35,2.33,-3.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,A teacher is explaining to her class how diffe...,97.0,-5.63,3.06,NaN,NaN,3.11,0.05,NaN,2.14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,Age and Womanhood 1. Between the ages of 13 an...,98.0,NaN,0.34,NaN,NaN,6.55,-9.08,NaN,-0.05,...,NaN,NaN,NaN,NaN,-0.24,NaN,NaN,NaN,NaN,NaN
99,A bus station is where a bus stops. A train st...,99.0,NaN,-4.32,NaN,NaN,1.80,-5.05,NaN,1.31,...,NaN,NaN,NaN,9.81,NaN,NaN,NaN,NaN,NaN,NaN


## GENERATE JOKES POPULARITY

In [6]:
#GENERATE POPULARITY BASED ON THE AVERAGE RATINGS PER JOKE
jokes_rate_df['Popularity'] = jokes_rate_df.iloc[:,2:].mean(axis=1)
jokes_rate_df

,Jokes,joke_id,User1,User2,User3,User4,User5,User6,User7,User8,...,User73413,User73414,User73415,User73416,User73417,User73418,User73419,User73420,User73421,Popularity
1,"A man visits the doctor. The doctor says ""I ha...",1.0,-7.82,4.08,NaN,NaN,8.50,-6.17,NaN,6.84,...,NaN,NaN,7.14,6.46,NaN,NaN,NaN,NaN,NaN,0.901997
2,This couple had an excellent relationship g...,2.0,8.79,-0.29,NaN,8.35,4.61,-3.54,NaN,3.16,...,NaN,NaN,NaN,8.69,NaN,NaN,NaN,NaN,NaN,0.162989
3,Q. What's 200 feet long and has 4 teeth? A...,3.0,-9.66,6.36,NaN,NaN,-4.17,0.44,NaN,9.17,...,NaN,NaN,3.88,9.08,NaN,NaN,NaN,NaN,NaN,0.193411
4,Q. What's the difference between a man and ...,4.0,-8.16,4.37,NaN,NaN,-5.39,-8.50,NaN,-6.21,...,NaN,NaN,1.12,3.79,NaN,NaN,NaN,NaN,NaN,-1.412599
5,Q.\tWhat's O. J. Simpson's Internet address...,5.0,-7.52,-2.38,9.03,1.80,1.36,-7.09,8.59,-8.16,...,NaN,8.88,NaN,6.46,5.78,5.24,-3.93,9.42,0.19,0.235352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Two attorneys went into a diner and ordered tw...,96.0,NaN,-2.14,NaN,NaN,1.55,-3.35,2.33,-3.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.373027
97,A teacher is explaining to her class how diffe...,97.0,-5.63,3.06,NaN,NaN,3.11,0.05,NaN,2.14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.489106
98,Age and Womanhood 1. Between the ages of 13 an...,98.0,NaN,0.34,NaN,NaN,6.55,-9.08,NaN,-0.05,...,NaN,NaN,NaN,-0.24,NaN,NaN,NaN,NaN,NaN,0.869864
99,A bus station is where a bus stops. A train st...,99.0,NaN,-4.32,NaN,NaN,1.80,-5.05,NaN,1.31,...,NaN,NaN,9.81,NaN,NaN,NaN,NaN,NaN,NaN,-0.036189


In [7]:
jokes_rate_df = jokes_rate_df.drop(columns=[col for col in jokes_rate_df.columns if 'User' in col])
jokes_rate_df

,Jokes,joke_id,Popularity
1,"A man visits the doctor. The doctor says ""I ha...",1.0,0.901997
2,This couple had an excellent relationship g...,2.0,0.162989
3,Q. What's 200 feet long and has 4 teeth? A...,3.0,0.193411
4,Q. What's the difference between a man and ...,4.0,-1.412599
5,Q.\tWhat's O. J. Simpson's Internet address...,5.0,0.235352
...,...,...,...
96,Two attorneys went into a diner and ordered tw...,96.0,1.373027
97,A teacher is explaining to her class how diffe...,97.0,1.489106
98,Age and Womanhood 1. Between the ages of 13 an...,98.0,0.869864
99,A bus station is where a bus stops. A train st...,99.0,-0.036189


## JOKES TEXT PREPROCESSING

In [8]:
stop_words = nltk.corpus.stopwords.words('english')
def normalize_document(doc):
    doc = doc.lower()
    doc = doc.strip()
    #tokens = nltk.word_tokenize(doc)
    #filtered_tokens = [token for token in tokens if token not in stop_words]
    #doc = ' '.join(filtered_tokens)
    return doc

In [9]:
normalize_corpus = np.vectorize(normalize_document)
norm_corpus = normalize_corpus(list(jokes_rate_df['Jokes']))
len(norm_corpus)

100

## EXTRACT TF-IDF FEATURES

In [10]:
tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(100, 766)

## COSINE SIMILARITY FOR PAIRWISE DOCUMENT SIMILARITY

In [11]:
doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,0.046421,0.028030,0.102968,0.000000,0.080318,0.000000,0.032025,0.065909,0.040910,...,0.070606,0.067675,0.086619,0.088824,0.080095,0.038815,0.021024,0.032057,0.045136,0.115008
1,0.046421,1.000000,0.017757,0.034254,0.054770,0.169652,0.032412,0.100444,0.087941,0.061116,...,0.087921,0.096184,0.098798,0.036582,0.030542,0.086396,0.093963,0.110528,0.030426,0.046018
2,0.028030,0.017757,1.000000,0.061987,0.218650,0.050234,0.000000,0.013364,0.075656,0.017065,...,0.107365,0.013453,0.082302,0.031785,0.011567,0.088924,0.011298,0.087104,0.000000,0.050225
3,0.102968,0.034254,0.061987,1.000000,0.152969,0.073805,0.020846,0.042227,0.087144,0.044155,...,0.159653,0.039703,0.106930,0.041785,0.051765,0.055420,0.047963,0.144315,0.000000,0.255327
4,0.000000,0.054770,0.218650,0.152969,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.044009,0.000000,0.102035,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [12]:
jokes_list = jokes_rate_df['Jokes'].values
jokes_list, jokes_list.shape

(array(['A man visits the doctor. The doctor says "I have bad news for you.You have cancer and Alzheimer\'s disease".  The man replies "Well,thank God I don\'t have cancer!"',
        '   This couple had an excellent relationship going until one day he came home from work to find his girlfriend packing. He asked her why she was leaving him and she told him that she had heard awful things about him.  "What could they possibly have said to make you move out?"  "They told me that you were a pedophile."  He replied, "That\'s an awfully big word for a ten year old." ',
        "   Q. What's 200 feet long and has 4 teeth?  A. The front row at a Willie Nelson Concert.",
        "   Q. What's the difference between a man and a toilet?  A. A toilet doesn't follow you around after you use it.",
        "   Q.\tWhat's O. J. Simpson's Internet address?  A.\tSlash, slash, backslash, slash, slash, escape.",
        "   Bill & Hillary are on a trip back to Arkansas. They're almost out of gas, so Bill

## FIND TOP SIMILAR JOKES FOR A SAMPLE JOKE

In [13]:
#FIND JOKE ID
jokes_idx = np.where(jokes_list == '   Q. What is orange and sounds like a parrot?   A. A carrot.')[0][0]
jokes_idx



15

In [14]:
#GET JOKE SIMILARITIES
jokes_similarities = doc_sim_df.iloc[jokes_idx].values
jokes_similarities

array([0.0128604 , 0.02690312, 0.10740195, 0.07513903, 0.33126323,
       0.03196095, 0.        , 0.        , 0.1015907 , 0.02585448,
       0.06609122, 0.09326017, 0.        , 0.08246224, 0.04686935,
       1.        , 0.        , 0.00948932, 0.10026069, 0.07549685,
       0.06682763, 0.16585176, 0.28017454, 0.05448154, 0.07240918,
       0.01809141, 0.02546236, 0.02675435, 0.0466826 , 0.08427484,
       0.13897486, 0.00980208, 0.05334661, 0.        , 0.01817428,
       0.03272277, 0.14037911, 0.        , 0.17722703, 0.01508852,
       0.05309758, 0.04445125, 0.0203685 , 0.        , 0.04368265,
       0.02260533, 0.04258376, 0.        , 0.        , 0.02012027,
       0.05453336, 0.06375007, 0.08353081, 0.08914284, 0.04393916,
       0.09794037, 0.        , 0.        , 0.36565019, 0.04154714,
       0.02506255, 0.11677163, 0.0938638 , 0.40987904, 0.05321105,
       0.02389569, 0.04574508, 0.05512095, 0.05730067, 0.03637326,
       0.01063885, 0.00760951, 0.21032537, 0.03081752, 0.02074

In [15]:
#GET TOP 5 SIMILAR JOKES IDs
similar_jokes_idxs = np.argsort(-jokes_similarities)[1:6]
similar_jokes_idxs

array([97, 63, 58,  4, 22])

In [16]:
#GET TOP 5 SIMILAR JOKES
similar_jokes = jokes_list[similar_jokes_idxs]
similar_jokes

array(["Age and Womanhood 1. Between the ages of 13 and 18 ... \xa0\xa0\xa0\xa0She is like Africa, virgin and unexplored. 2. Between the ages of 19 and 35 ... \xa0\xa0\xa0\xa0She is like Asia, hot and exotic. 3. Between the ages of 36 and 45 ... \xa0\xa0\xa0\xa0She is like America, fully explored, breathtakingly beautiful,and free with her resources. 4. Between the ages of 46 and 56 ... \xa0\xa0\xa0\xa0She is like Europe, exhausted but still has points of interest. 5. After 56 she is like Australia ... \xa0\xa0\xa0\xa0Everybody knows it's down there, but who gives a damn?",
       '   What is the rallying cry of the International Dyslexic Pride movement? Dyslexics Untie!',
       '   The Chukcha (Russian Eskimo) phones up the Russian Parliament Building.   A guard answers.  Chukcha:  "What is required to become Parliament member?" Guard:  "What are you, an idiot?" Chukcha:  "Is it required?"',
       "   Q.\tWhat's O. J. Simpson's Internet address?  A.\tSlash, slash, backslash, slash, 

## BUILD A JOKE CONTENT-BASED RECOMMENDER

In [17]:
#RECOMMENDATION FUNCTION
def jokes_recommender(joke, jokes=jokes_list, doc_sims=doc_sim_df):
    joke_idx = np.where(jokes_list == joke)[0][0]
    joke_similarities = doc_sims.iloc[joke_idx].values
    similar_joke_idxs = np.argsort(-joke_similarities)[1:6]
    similar_jokes = jokes[similar_joke_idxs]
    return similar_jokes

## GET A LIST OF MOST POPULAR JOKES

In [19]:
popular_jokes = jokes_rate_df.sort_values(by='Popularity', ascending=False)
popular_jokes

,Jokes,joke_id,Popularity
50,A guy goes into confession and says to the ...,50.0,3.362593
89,A radio conversation of a US naval ship with ...,89.0,3.348026
32,A man arrives at the gates of heaven. St. P...,32.0,2.974698
36,"A guy walks into a bar, orders a beer and s...",36.0,2.917245
27,Clinton returns from a vacation in Arkansas...,27.0,2.907989
...,...,...,...
13,They asked the Japanese visitor if they hav...,13.0,-1.944859
44,"A horse walks into a bar. Bartender says: ""...",44.0,-1.998306
15,Q: What did the blind person say when give...,15.0,-2.112506
16,Q. What is orange and sounds like a parrot?...,16.0,-3.103138


In [20]:
top_five_popular_jokes = popular_jokes['Jokes'].head().tolist()
top_five_popular_jokes

['   A guy goes into confession and says to the priest, "Father, I\'m 80 years old, widower, with 11 grandchildren. Last night I met two beautiful flight attendants. They took me home and I made love to both of them. Twice." The priest said: "Well, my son, when was the last time you were in confession?"  "Never Father, I\'m Jewish."  "So then, why are you telling me?"  "I\'m telling everybody."',
 "A radio conversation of a US naval  ship with Canadian authorities ...  Americans: Please divert your course 15 degrees to the North to avoid a collision. Canadians: Recommend you divert YOUR course 15 degrees to the South to  avoid a collision. Americans: This is the Captain of a US Navy ship.  I say again, divert  YOUR course. Canadians: No.  I say again, you divert YOUR course. Americans: This is the aircraft carrier USS LINCOLN, the second largest ship in the United States' Atlantic Fleet. We are accompanied by three destroyers, three cruisers and numerous support vessels. I demand that 

## RECOMMENDATION RESULT

In [22]:
for joke in top_five_popular_jokes:
    print('JOKE:', joke)
    print('TOP 5 RECOMMENDED JOKES:', jokes_recommender(joke))
    print('###########################')
    print()

JOKE:    A guy goes into confession and says to the priest, "Father, I'm 80 years old, widower, with 11 grandchildren. Last night I met two beautiful flight attendants. They took me home and I made love to both of them. Twice." The priest said: "Well, my son, when was the last time you were in confession?"  "Never Father, I'm Jewish."  "So then, why are you telling me?"  "I'm telling everybody."
TOP 5 RECOMMENDED JOKES: ['   A country guy goes into a city bar that has a dress code, and the maitre d\'  demands he wear a tie. Discouraged, the guy goes to his car to sulk when  inspiration strikes: He\'s got jumper cables in the trunk! So he wraps them around his neck, sort of like a string tie (a bulky string tie to be sure) and returns to the bar. The maitre d\' is reluctant, but says to the guy, "Okay, you\'re a pretty resourceful fellow, you can come in... but just don\'t start anything"!  '
 '   A guy walks into a bar, orders a beer and says to the bartender, "Hey, I got this great Po